# Import libraries

In [1]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd

# Pull list of teams from website homepage

Start with the initial url of the homepage for baseball reference.

In [2]:
url = 'https://www.baseball-reference.com'

In [3]:
#pull html from the above url
page_html = re.get(url)
#turn the html into a beautiful soup object
soup = bs(page_html.text)

In [4]:
#find all the tags for the teams 
teams = soup.find_all('div',class_='division')

# Create list of urls for each team with their current roster

In [14]:
year = '2021'
new_list = []
#teams is a list of the initials of all teams
for division in teams:
    team_names = division.find_all('a')
    #creates the url with the abbreviation for each team
    for links in team_names:
        final_link = links['href'].replace('/teams','')
        #adds details to the url 
        final_link = url + '/teams/'+ final_link + year + '.shtml'
        new_list.append(final_link)

In [45]:
new_list

['https://www.baseball-reference.com/teams/ATL/2021.shtml',
 'https://www.baseball-reference.com/teams/FLA/2021.shtml',
 'https://www.baseball-reference.com/teams/NYM/2021.shtml',
 'https://www.baseball-reference.com/teams/PHI/2021.shtml',
 'https://www.baseball-reference.com/teams/WSN/2021.shtml',
 'https://www.baseball-reference.com/teams/CHC/2021.shtml',
 'https://www.baseball-reference.com/teams/CIN/2021.shtml',
 'https://www.baseball-reference.com/teams/MIL/2021.shtml',
 'https://www.baseball-reference.com/teams/PIT/2021.shtml',
 'https://www.baseball-reference.com/teams/STL/2021.shtml',
 'https://www.baseball-reference.com/teams/ARI/2021.shtml',
 'https://www.baseball-reference.com/teams/COL/2021.shtml',
 'https://www.baseball-reference.com/teams/LAD/2021.shtml',
 'https://www.baseball-reference.com/teams/SDP/2021.shtml',
 'https://www.baseball-reference.com/teams/SFG/2021.shtml',
 'https://www.baseball-reference.com/teams/BAL/2021.shtml',
 'https://www.baseball-reference.com/tea

# Pull Player URLs from Team Home Page

In [81]:
#this function takes in a team url and returns a list of player urls that appear on that page
def player_links_from_team(team_url):
    raw_data = re.get(team_url)
    html_of_team_page = bs(raw_data.text)
    #in this analysis we are only pulling position players so there is no pitchers
    batting_table = html_of_team_page.find('div',id='div_team_batting')
    all_players = batting_table.find('tbody').find_all('tr')
    #initialize list for players urls
    list_of_player_links = []
    for i in all_players:
        position = i.find(attrs={'data-stat':'pos'}).text
        #filter pitchers out before adding them to the list of player links
        if position != 'P' and position != 'Pos':
            player_link = i.find('a')['href']
            #add the player extension to the standard webpage before adding to list_of_player_links
            list_of_player_links.append(url+player_link)
    return list_of_player_links

# Pull Stats from Player Link

In [10]:
#this function takes in a player's URL and then outputs a list of dictionaries, where each dictionary is a year of statistics
def data_pull_player_link(player_link):
    raw_data = re.get(player_link)
    html_of_player_page = bs(raw_data.text)  
    #there are three batting tables associated with each player
    standard_batting = html_of_player_page.find('div',id='div_batting_standard')
    #the other tables are commented out in the html, so it is necessary to remove the <!-- symbols in order to see these tables
    html_of_player_page_string = str(html_of_player_page)
    #removing these comments
    removing_comments = html_of_player_page_string.replace('<!--','').replace('-->','')
    #changing back to Beautiful Soup
    html_without_comments = bs(removing_comments)
    #second table of statistics
    player_value_batting = html_without_comments.find('div',id='div_batting_value').find('tbody').find_all('tr',class_=['full','full rowSum'])
    #try to pull the third table of statistics if available
    try:
        advanced_batting = html_without_comments.find('div',id='div_batting_advanced').find('tbody').find_all('tr',class_=['partial_table','full'])
    except:
        advanced_batting = []
    standard_batting_lines = standard_batting.find('tbody').find_all('tr',class_=['partial_table','full'])
    #initialize lists for cleaned data
    list_of_years = []
    new_standard = []
    new_advanced = []
    #remove totaled rows to avoid duplicate data in standard Batting and advanced batting
    for i in standard_batting_lines:
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT' and i.find('td',attrs={'data-stat':'team_ID'}).text != '':
            new_standard.append(i)
    for i in advanced_batting:
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT'and i.find('td',attrs={'data-stat':'team_ID'}).text != '':
            new_advanced.append(i)
    #go through each year in the zip of all three tables
    for i,j,k in zip(new_standard,player_value_batting,new_advanced):
        #this filters out the TOTAL rows
        if i.find('td',attrs={'data-stat':'team_ID'}).text != 'TOT':
            #initialize dictionary for given year
            year_dict = {}
            #pull in data for each datapoint from one of the three tables
            year_dict['year'] = i.find('th',attrs={'data-stat':'year_ID'}).text
            year_dict['age'] = i.find('td',attrs={'data-stat':'age'}).text
            year_dict['team_ID'] = i.find('td',attrs={'data-stat':'team_ID'}).text
            year_dict['games'] = i.find('td',attrs={'data-stat':'G'}).text
            year_dict['PA'] = i.find('td',attrs={'data-stat':'PA'}).text
            year_dict['AB'] = i.find('td',attrs={'data-stat':'AB'}).text
            year_dict['R'] = i.find('td',attrs={'data-stat':'R'}).text
            year_dict['H'] = i.find('td',attrs={'data-stat':'H'}).text
            year_dict['2B'] = i.find('td',attrs={'data-stat':'2B'}).text
            year_dict['3B'] = i.find('td',attrs={'data-stat':'3B'}).text
            year_dict['HR'] = i.find('td',attrs={'data-stat':'HR'}).text
            year_dict['RBI'] = i.find('td',attrs={'data-stat':'RBI'}).text
            year_dict['SB'] = i.find('td',attrs={'data-stat':'SB'}).text
            year_dict['CS'] = i.find('td',attrs={'data-stat':'CS'}).text
            year_dict['BB'] = i.find('td',attrs={'data-stat':'BB'}).text
            year_dict['SO'] = i.find('td',attrs={'data-stat':'SO'}).text
            year_dict['batting_avg'] = i.find('td',attrs={'data-stat':'batting_avg'}).text
            year_dict['onbase_perc'] = i.find('td',attrs={'data-stat':'onbase_perc'}).text
            year_dict['slugging_perc'] = i.find('td',attrs={'data-stat':'slugging_perc'}).text
            year_dict['onbase_plus_slugging'] = i.find('td',attrs={'data-stat':'onbase_plus_slugging'}).text
            year_dict['onbase_plus_slugging_plus'] = i.find('td',attrs={'data-stat':'onbase_plus_slugging_plus'}).text
            year_dict['TB'] = i.find('td',attrs={'data-stat':'TB'}).text
            year_dict['GIDP'] = i.find('td',attrs={'data-stat':'GIDP'}).text
            year_dict['HBP'] = i.find('td',attrs={'data-stat':'HBP'}).text
            year_dict['SH'] = i.find('td',attrs={'data-stat':'SH'}).text
            year_dict['SF'] = i.find('td',attrs={'data-stat':'SF'}).text
            year_dict['IBB'] = i.find('td',attrs={'data-stat':'IBB'}).text
            year_dict['Pos'] = i.find('td',attrs={'data-stat':'pos_season'}).text
            year_dict['awards'] = i.find('td',attrs={'data-stat':'award_summary'}).text
            year_dict['runs_bat'] = j.find('td',attrs={'data-stat':'runs_bat'}).text
            year_dict['runs_br'] = j.find('td',attrs={'data-stat':'runs_br'}).text
            year_dict['runs_dp'] = j.find('td',attrs={'data-stat':'runs_dp'}).text
            year_dict['runs_fielding'] = j.find('td',attrs={'data-stat':'runs_fielding'}).text
            year_dict['runs_above_avg_bat'] = j.find('td',attrs={'data-stat':'runs_above_avg_bat'}).text
            year_dict['WAA'] = j.find('td',attrs={'data-stat':'WAA'}).text
            year_dict['runs_replacement'] = j.find('td',attrs={'data-stat':'runs_replacement'}).text
            year_dict['runs_above_rep'] = j.find('td',attrs={'data-stat':'runs_above_rep'}).text
            year_dict['WAR'] = j.find('td',attrs={'data-stat':'WAR'}).text
            year_dict['waa_win_perc'] = j.find('td',attrs={'data-stat':'waa_win_perc'}).text
            year_dict['waa_win_perc_162'] = j.find('td',attrs={'data-stat':'waa_win_perc_162'}).text
            year_dict['WAR_off'] = j.find('td',attrs={'data-stat':'WAR_off'}).text
            year_dict['WAR_def'] = j.find('td',attrs={'data-stat':'WAR_def'}).text
            year_dict['runs_above_rep_off'] = j.find('td',attrs={'data-stat':'runs_above_rep_off'}).text
            year_dict['Salary'] = j.find('td',attrs={'data-stat':'Salary'}).text
            year_dict['rOBA'] = k.find('td',attrs={'data-stat':'rOBA'}).text
            year_dict['Rbat+'] = k.find('td',attrs={'data-stat':'Rbat+'}).text
            year_dict['adv_bat_babip'] = k.find('td',attrs={'data-stat':'adv_bat_babip'}).text
            year_dict['adv_bat_iso'] = k.find('td',attrs={'data-stat':'adv_bat_iso'}).text
            year_dict['adv_bat_home_run_perc'] = k.find('td',attrs={'data-stat':'adv_bat_home_run_perc'}).text
            year_dict['adv_bat_strikeout_perc'] = k.find('td',attrs={'data-stat':'adv_bat_strikeout_perc'}).text
            year_dict['adv_bat_base_on_balls_perc'] = k.find('td',attrs={'data-stat':'adv_bat_base_on_balls_perc'}).text
            year_dict['adv_bat_exit_velo'] = k.find('td',attrs={'data-stat':'adv_bat_exit_velo'}).text
            year_dict['adv_bat_hard_hit_perc'] = k.find('td',attrs={'data-stat':'adv_bat_hard_hit_perc'}).text
            year_dict['adv_bat_ld_perc'] = k.find('td',attrs={'data-stat':'adv_bat_ld_perc'}).text
            year_dict['adv_bat_gb_perc'] = k.find('td',attrs={'data-stat':'adv_bat_gb_perc'}).text
            year_dict['adv_bat_fb_perc'] = k.find('td',attrs={'data-stat':'adv_bat_fb_perc'}).text
            year_dict['adv_bat_gb_fb_ratio'] = k.find('td',attrs={'data-stat':'adv_bat_gb_fb_ratio'}).text
            year_dict['adv_bat_pull_perc'] = k.find('td',attrs={'data-stat':'adv_bat_pull_perc'}).text
            year_dict['adv_bat_cent_perc'] = k.find('td',attrs={'data-stat':'adv_bat_cent_perc'}).text
            year_dict['adv_bat_oppo_perc'] = k.find('td',attrs={'data-stat':'adv_bat_oppo_perc'}).text
            year_dict['adv_bat_wpa_bat'] = k.find('td',attrs={'data-stat':'adv_bat_wpa_bat'}).text
            year_dict['adv_bat_cwpa_bat'] = k.find('td',attrs={'data-stat':'adv_bat_cwpa_bat'}).text
            year_dict['adv_bat_re24_bat'] = k.find('td',attrs={'data-stat':'adv_bat_re24_bat'}).text
            #add dictionary to final list
            list_of_years.append(year_dict)              
    #return final list of dictionaries
    return list_of_years
    

# Turn a list of players into a DataFrame

In [11]:
#list function iterates through a list of players and return a dataframe of all players in the list
def list_of_players_to_df(list_of_players):
    final_dataframe = []
    for i in list_of_players:
        #calls previous function to pull stats for each player
        stats_of_player = data_pull_player_link(i)
        for j in stats_of_player:
            final_dataframe.append(j)
    return final_dataframe

# Builds DataFrame from list of teams

In [12]:
#this function takes a list of teams and return a dataframe for all players on each team
def final_dataframe_builder(list_of_teams):
    final_dataframe = []
    for i in list_of_teams:
        #retrieves a list of all players on a given team
        list_of_players_ = player_links_from_team(i)
        #returns Dataframe from list of players
        answer = list_of_players_to_df(list_of_players_)
        for k in answer:
            final_dataframe.append(k)
    return final_dataframe

In [15]:
#splits each list into a division
nl_east = new_list[0:5]
nl_central = new_list[5:10]
nl_west = new_list[10:15]
al_east = new_list[15:20]
al_central = new_list[20:25]
al_west = new_list[25:30]

# Creates Dataframes for Each Division

In [522]:
nl_east_df = pd.DataFrame(final_dataframe_builder(nl_east))

In [524]:
nl_central_df = pd.DataFrame(final_dataframe_builder(nl_central))

In [525]:
nl_west_df = pd.DataFrame(final_dataframe_builder(nl_west))

In [528]:
al_east_df = pd.DataFrame(final_dataframe_builder(al_east))

In [529]:
al_central_df = pd.DataFrame(final_dataframe_builder(al_central))

In [541]:
al_west_df = pd.DataFrame(final_dataframe_builder(al_west))

# Combines DataFrames for Each Division

In [543]:
final_df = pd.concat([nl_east_df,nl_central_df,nl_west_df,al_east_df,al_central_df,al_west_df])

In [546]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4163 entries, 0 to 669
Data columns (total 63 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   year                        4163 non-null   object
 1   age                         4163 non-null   object
 2   team_ID                     4163 non-null   object
 3   games                       4163 non-null   object
 4   PA                          4163 non-null   object
 5   AB                          4163 non-null   object
 6   R                           4163 non-null   object
 7   H                           4163 non-null   object
 8   2B                          4163 non-null   object
 9   3B                          4163 non-null   object
 10  HR                          4163 non-null   object
 11  RBI                         4163 non-null   object
 12  SB                          4163 non-null   object
 13  CS                          4163 non-null   objec

In [547]:
final_df.to_csv('baseball_players.csv')